# The Graph Data Access

In this notebook, we retrieve live Rai system data from [The Graph](https://thegraph.com/explorer/subgraph/reflexer-labs/rai-mainnet?query=Example%20query) and save it as a CSV for analysis in the [Graph Analysis](TheGraphDataAnalysis.ipynb) notebook.

In [1]:
# import libraries
import pandas as pd
import json
import requests
import matplotlib.pyplot as plt
url = 'https://api.thegraph.com/subgraphs/name/reflexer-labs/rai-mainnet'

## Hourly stats

In [2]:
query_header = '''
query {{
    hourlyStats(first: 1000, skip:{}) {{'''

query_tail = '''    
}
}'''

query_body = '''
 timestamp
    blockNumber
    redemptionPrice {
      id
      block
      redemptionRate
      value
      
    }
    marketPriceUsd
    marketPriceEth
    globalDebt
    erc20CoinTotalSupply
'''





n = 0
safes = []
while True:
    print(f'request {n+1}')
    query = query_header.format(n*1000) + query_body + query_tail
    r = requests.post(url, json = {'query':query})
    s = json.loads(r.content)['data']['hourlyStats']
    print(f'results {len(s)}')
    safes.extend(s)
    print(f'total {len(safes)}')
    n += 1
    if len(s) < 1000:
        break
        
hourlyStats = pd.DataFrame(safes)

request 1
results 555
total 555


In [3]:
hourlyStats

,blockNumber,erc20CoinTotalSupply,globalDebt,marketPriceEth,marketPriceUsd,redemptionPrice,timestamp
0,11857308,1988.440432971667032725,1988.577499489843641169436691184972,0.002589462210048543650949087456484599,4.670500166535429740591356399938391,"{'block': '11849022', 'id': '0xe313271188492c4...",1613338681
1,11860793,2788.440432971667032725,2788.617717345781163523077653358556,0.002481355746535171105569862943787003,4.377049344386119132419461918241039,"{'block': '11860755', 'id': '0x40066774f6fb7d3...",1613384616
2,11861040,2788.440432971667032725,2788.657935201718685876567866781341,0.002481355746535171105569862943787003,4.406833024856393396658172078078713,"{'block': '11861008', 'id': '0x0d551e2a2d55fa8...",1613387890
3,11861337,2788.440432971667032725,2788.657935201718685876567866781341,0.002481355746535171105569862943787003,4.419265710853707004204445293455575,"{'block': '11861237', 'id': '0x5c67dbb323db039...",1613392101
4,11861474,3048.440432971667032725,3048.645954953510904626288422086589,0.001768934955701819449591545208406114,3.194329891923853932595347583210722,"{'block': '11861237', 'id': '0x5c67dbb323db039...",1613394096
...,...,...,...,...,...,...,...
550,12010465,42476013.82242584163675588,43314023.31815621287656032719477108,0.001625041400702411519611518995187305,2.969551544064885546839771107577204,"{'block': '12010443', 'id': '0x997691a78d9c595...",1615374937
551,12010817,43298317.625212681564702321,44136428.85174739846689061297976533,0.001636721874314180917398266446809564,2.96986173023419900041053010131036,"{'block': '12010709', 'id': '0xdcc1f29b7a3b611...",1615379588
552,12010941,43399117.625212681564702321,44237308.41661038906508775402546272,0.001636721874314180917398214033725258,2.976424949253294787096916026631926,"{'block': '12010709', 'id': '0xdcc1f29b7a3b611...",1615381309
553,12011285,43447340.624047229286881326,44285585.09229244328704527080606708,0.001626771089120311202244328025043454,2.988082185910203146591551313589792,"{'block': '12011245', 'id': '0xb6a01db82df1fdd...",1615385895


In [4]:
hourlyStats.redemptionPrice.values[0]

{'block': '11849022',
 'id': '0xe313271188492c4cb881c814c2ce2797f638c56c4f0198abbfc7da11d9b06ed3-7',
 'redemptionRate': '1',
 'value': '3.14'}

In [5]:
hourlyStats['redemptionPriceActual'] = hourlyStats.redemptionPrice.apply(lambda x: x['value'])

In [6]:
hourlyStats['redemptionRateActual'] = hourlyStats.redemptionPrice.apply(lambda x: x['redemptionRate'])

In [7]:
hourlyStats['erc20CoinTotalSupply'] = hourlyStats['erc20CoinTotalSupply'].astype(float)
hourlyStats['globalDebt'] = hourlyStats['globalDebt'].astype(float)
hourlyStats['blockNumber'] = hourlyStats['blockNumber'].astype(int)
hourlyStats['marketPriceEth'] = hourlyStats['marketPriceEth'].astype(float)
hourlyStats['marketPriceUsd'] = hourlyStats['marketPriceUsd'].astype(float)
hourlyStats['redemptionPriceActual'] = hourlyStats['redemptionPriceActual'].astype(float)
hourlyStats['redemptionRateActual'] = hourlyStats['redemptionRateActual'].astype(float)

In [8]:
hourlyStats.describe()

,blockNumber,erc20CoinTotalSupply,globalDebt,marketPriceEth,marketPriceUsd,redemptionPriceActual,redemptionRateActual
count,5.550000e+02,5.550000e+02,5.550000e+02,555.000000,555.000000,555.000000,5.550000e+02
mean,1.193633e+07,2.605970e+07,2.658475e+07,0.001999,3.335829,3.091356,1.000000e+00
std,4.359907e+04,1.258564e+07,1.293412e+07,0.000313,0.512416,0.042294,1.522526e-08
min,1.185731e+07,1.988440e+03,1.988577e+03,0.001609,2.850455,3.016181,9.999999e-01
25%,1.189869e+07,1.591626e+07,1.593316e+07,0.001785,3.152961,3.049830,1.000000e+00
50%,1.193632e+07,3.273819e+07,3.370695e+07,0.001971,3.226861,3.106774,1.000000e+00
75%,1.197415e+07,3.446885e+07,3.545641e+07,0.002119,3.325158,3.129599,1.000000e+00
max,1.201161e+07,4.391989e+07,4.476692e+07,0.004474,8.055382,3.140000,1.000000e+00


In [9]:
hourlyStats.to_csv('saved_results/RaiLiveGraphData.csv')

## Collateral Prices

## Safes

## Conclusion

Using The Graph, a lot of data about the Rai system can be obtained for analyzing the health of the system. With some data manipulation, these data streams could be intergrated into the Rai cadCAD model to turn it into a true decision support system.